In [ ]:
import numpy as np
import pandas as pd

#import scipy
import matplotlib.pyplot as plt
#import os
#from scipy import io

import seaborn as sns
#import feather

#import pysubgroup as ps

#from scipy import stats
#from scipy.stats import zscore
#from sklearn.preprocessing import StandardScaler
#from sklearn import preprocessing
#from sklearn.model_selection import train_test_split

#from sklearn.cluster import KMeans
#from sklearn import preprocessing

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
# Read Head
df_eban = pd.read_excel('../Data1/EBAN.XLSX')

In [ ]:
df_eban.head(4)

In [ ]:
#df_bseg = pd.read_excel('../Data3/bseg1.XLSX')

In [ ]:
print(df_eban.shape)


# Data Cleansing

## initial cleaning

In [ ]:
df = df_eban.copy()

In [ ]:

# Spalten die nur NAN enthalten entfernen
df = df.dropna(axis=1, how='all')


# Spalten die ausschließlich XY Werte enthalten (0 / X / 1 / etc.) dropen
df = df.loc[:, (df.astype(str) != 0).any(axis=0)]
df = df.loc[:, (df.astype(str) != '0,0').any(axis=0)]
df = df.loc[:, (df.astype(str) != '0,00').any(axis=0)]
df = df.loc[:, (df.astype(str) != '0,000').any(axis=0)]
df = df.loc[:, (df.astype(str) != 'X').any(axis=0)]
df = df.loc[:, (df.astype(str) != '1').any(axis=0)]



#spaletn entfernen die immer die selben Werte enthalten
#nunique = df.apply(pd.Series.nunique)
#cols_to_drop = nunique[nunique == 1].index
#df = df.drop(cols_to_drop, axis=1)
#df.drop(columns=df.columns[df.nunique()==1], inplace=True)

keep_columns = [col for col in df.columns if len(df[col].unique()) > 1]
df = df[keep_columns].copy()


#print(len(df_io.index))
print(df.shape)

In [ ]:
# we looose many features here, hence data unity. 
# add time and date

In [ ]:
df.sample(10)

# Delete plant identifiers.

In [ ]:
df.drop(['EKORG'],axis=1, inplace=True)

In [ ]:
df.drop(['WERKS'],axis=1, inplace=True)

In [ ]:
df.drop(['SPRAS'],axis=1, inplace=True)

In [ ]:
df.MATKL = df.MATKL.str.slice(3)
df.MATNR = df.MATNR.str.slice(3)

In [ ]:
df.dtypes

## data types correction

In [ ]:
df.describe(include='all').T

In [ ]:
for (columnName, columnData) in df.items():
    print('Colunm Name : ', columnName )
    print('Colunm Name : ', df[columnName].dtypes )
    print('Number of unique values : ', df[columnName].nunique() ) 
    print('Column unique values : ', np.unique(df[columnName].astype(str)) )
    print('######')

In [ ]:
df.sample(4)

In [ ]:
df.shape

In [ ]:
cont_Cols = ['BANFN']

cat_Cols = ['EKGRP', 'TXZ01', 'MATNR', 'MATKL', 'MEINS', 'FLIEF', 'EBELN', 'EBELP']

num_Cols = ['MENGE', 'PREIS', 'BSMNG']

dat_Cols = ['ERDAT', 'BADAT', 'BEDAT', 'LFDAT', 'FRGDT']

#2 Del for Clustering
#EBELN 	EBELP 	

In [ ]:
#sanity check of cols
print(len(cont_Cols)+len(cat_Cols)+len(num_Cols)+len(dat_Cols))
print(df.shape[1]) 

In [ ]:
# Change dtype to Category for relevant features
df_cor = df.copy()

# we have to change NaN top 0 to avoid the infer dtype "object" for some cols
#df_cor['ZUONR'] = df_cor['ZUONR'].fillna(0)


# change Dtype to Cat for categorical columns
cats_cat = df_cor[cat_Cols]
        
for (columnName, columnData) in cats_cat.items():
    df_cor[columnName] = df_cor[columnName].astype(str)
    if isinstance(df_cor[columnName][0], str):
        s = pd.Series(df_cor[columnName], dtype="category")
        df_cor[columnName] = df_cor[columnName].astype('category')


In [ ]:
#check 1
df_cor.dtypes

In [ ]:
#check 2
print('Categorical columns: {}'.format(list(df_cor.select_dtypes('category').columns)))

In [ ]:
#check 3
df_cor.shape

In [ ]:
import missingno as msno
%matplotlib inline
msno.matrix(df_cor)

# EXPORT 1 cleaned data with the right dtypes but still with correlating variables

In [ ]:
df_cor.to_feather('../Data/D3_EBAN_raw.ftr')

# lets check the dataset properities / Correlation  / distribution

In [ ]:
# Compute the corr matrix, only for numerics since we use category dtype
corr = df_cor.corr(numeric_only=True)

# Set up the triangular mask / matplotlib figure / Colors
mask = np.triu(np.ones_like(corr, dtype=bool)) # tril for upper
f, ax = plt.subplots(figsize=(4, 4))
cmap1 = sns.cubehelix_palette(dark=0, light=1, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap1, vmax=1,vmin=-1, center=0, square=True, annot=True, linewidths=.5, cbar_kws={"shrink": .5})


In [ ]:

# Compute the corr matrix and 
corr = df.corr(numeric_only=True).round(2)

# Set up the triangular mask / matplotlib figure / Colors
mask = np.triu(np.ones_like(corr, dtype=bool)) # tril for upper
f, ax = plt.subplots(figsize=(7, 7))
cmap1 = sns.cubehelix_palette(dark=0, light=1, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap1, vmax=1,vmin=-1, center=0, square=True, annot=True, linewidths=.5, cbar_kws={"shrink": .5})


In [ ]:
df_cor.head(10)

In [ ]:
df.head(10)

In [ ]:
#manually delete the hihgly correlating features in both dataframes
# WRBTR PSWBT VBEL2 POSN2 ERFMG AUFPL SAKNR SAKNR LOKKT KIDNO
df = df.drop(['BSMNG'], axis=1)
df_cor = df_cor.drop(['BSMNG'], axis=1)

In [ ]:

# Compute the corr matrix and 
corr = df.corr(numeric_only=True).round(2)

# Set up the triangular mask / matplotlib figure / Colors
mask = np.triu(np.ones_like(corr, dtype=bool)) # tril for upper
f, ax = plt.subplots(figsize=(7, 7))
cmap1 = sns.cubehelix_palette(dark=0, light=1, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap1, vmax=1,vmin=-1, center=0, square=True, annot=True, linewidths=.5, cbar_kws={"shrink": .5})


In [ ]:
df_cor.to_feather('../Data/D3_EBAN_cleaned.ftr')